In [3]:
import sys
sys.path.append('../')

import numpy as np
from sklearn.datasets import make_circles
import HuichuanFlow as ms

X, y = make_circles(600, noise=0.1, factor=0.2)
y = y * 2 - 1

# 特征维数
dimension = 20

# 构造噪声特征
X = np.concatenate([X, np.random.normal(0.0, 0.5, (600, dimension-2))], axis=1)

# 隐藏向量维度
k = 2

# 一次项
x1 = ms.core.Variable(dim=(dimension, 1), init=False, trainable=False)

# 标签
label = ms.core.Variable(dim=(1, 1), init=False, trainable=False)

# 一次项权值向量
w = ms.core.Variable(dim=(1, dimension), init=True, trainable=True)

# 隐藏向量矩阵
H = ms.core.Variable(dim=(k, dimension), init=True, trainable=True)
HTH = ms.ops.MatMul(ms.ops.Reshape(H, shape=(dimension, k)), H)

# 偏置
b = ms.core.Variable(dim=(1, 1), init=True, trainable=True)

# 线性部分
output = ms.ops.Add(
        ms.ops.MatMul(w, x1),   # 一次部分
        
        # 二次部分
        ms.ops.MatMul(ms.ops.Reshape(x1, shape=(1, dimension)),
                      ms.ops.MatMul(HTH, x1)),
        b)

# 预测概率
predict = ms.ops.Logistic(output)

# 损失函数
loss = ms.ops.loss.LogLoss(ms.ops.MatMul(label, output))

learning_rate = 0.001
optimizer = ms.optimizer.Adam(ms.default_graph, loss, learning_rate)


batch_size = 16

for epoch in range(50):
    
    batch_count = 0   
    for i in range(len(X)):
        
        x1.set_value(np.mat(X[i]).T)
        label.set_value(np.mat(y[i]))
        
        optimizer.one_step()
        
        batch_count += 1
        if batch_count >= batch_size:
            
            optimizer.update()
            batch_count = 0
        

    pred = []
    for i in range(len(X)):
                
        x1.set_value(np.mat(X[i]).T)
        
        predict.forward()
        pred.append(predict.value[0, 0])
            
    pred = (np.array(pred) > 0.5).astype(np.int) * 2 - 1
    accuracy = (y == pred).astype(np.int).sum() / len(X)
       
    print("epoch: {:d}, accuracy: {:.3f}".format(epoch + 1, accuracy))

epoch: 1, accuracy: 0.548
epoch: 2, accuracy: 0.613
epoch: 3, accuracy: 0.647
epoch: 4, accuracy: 0.678
epoch: 5, accuracy: 0.685
epoch: 6, accuracy: 0.692
epoch: 7, accuracy: 0.707
epoch: 8, accuracy: 0.718
epoch: 9, accuracy: 0.720
epoch: 10, accuracy: 0.733
epoch: 11, accuracy: 0.743
epoch: 12, accuracy: 0.747
epoch: 13, accuracy: 0.752
epoch: 14, accuracy: 0.760
epoch: 15, accuracy: 0.773
epoch: 16, accuracy: 0.782
epoch: 17, accuracy: 0.788
epoch: 18, accuracy: 0.787
epoch: 19, accuracy: 0.795
epoch: 20, accuracy: 0.797
epoch: 21, accuracy: 0.803
epoch: 22, accuracy: 0.813
epoch: 23, accuracy: 0.818
epoch: 24, accuracy: 0.830
epoch: 25, accuracy: 0.835
epoch: 26, accuracy: 0.835
epoch: 27, accuracy: 0.840
epoch: 28, accuracy: 0.842
epoch: 29, accuracy: 0.845
epoch: 30, accuracy: 0.850
epoch: 31, accuracy: 0.853
epoch: 32, accuracy: 0.858
epoch: 33, accuracy: 0.857
epoch: 34, accuracy: 0.865
epoch: 35, accuracy: 0.865
epoch: 36, accuracy: 0.868
epoch: 37, accuracy: 0.873
epoch: 38,

In [4]:
print(H.value)

[[-0.19445295 -1.87596391  1.34071685 -1.23412641  0.33220487 -0.40328118
  -0.33026252  0.10348326  0.17097313  0.03204932  0.17384099 -0.11515193
  -0.03727271  0.04434071 -0.06973917  0.05023985  0.03314296 -0.02387814
   0.1094717   0.02968029]
 [ 1.07971851  1.08934038 -0.32733876 -0.11010544  0.16876109 -0.11554856
   0.14536421  0.02255521  0.07299045 -0.02203485 -0.03659781 -0.02497359
  -0.02391062 -0.06308496 -0.12254481 -0.00636658 -0.09143042 -0.04534507
   0.10432499  0.37336447]]


In [6]:
print(H.value.shape)

(2, 20)
